In [ ]:
#model imports
import numpy as np
import pandas as pd
try:
    from catboost import CatBoostRegressor, Pool
except ModuleNotFoundError:
    print("Installing catboost")
    !conda install catboost
    from catboost import CatBoostRegressor, Pool
from urllib.error import HTTPError
try:
    import nfl_data_py as nfl
except ModuleNotFoundError:
    print("Installing nfl_data_py")
    !pip install nfl_data_py
    import nfl_data_py as nfl
try:
    from espn_api.football import League
    from espn_api.football.constant import POSITION_MAP
except ModuleNotFoundError:
    print("Installing espn_api")
    !pip install espn_api
    from espn_api.football import League
    from espn_api.football.constant import POSITION_MAP
try:
    from sleeperpy import User,Leagues,Players
except ModuleNotFoundError:
    print("Installing sleeperpy")
    !pip install sleeper-py
    from sleeperpy import User,Leagues,Players
try:
    from yahoo_oauth import OAuth2
    import yahoo_fantasy_api as yfa
except ModuleNotFoundError:
    print("Installing yahoo_fantasy_api")
    !pip install yahoo_fantasy_api
    from yahoo_oauth import OAuth2
    import yahoo_fantasy_api as yfa

In [ ]:
def obtain_nfl_data():
    """
    Access relevant nfl data needed for the model. All data scraped from using nfl_py_data which uses NFL FastR.
    Input: 
    None
    
    Outputs:
    df_weekly_stats : Dataframe of weekly stats from NFL FastR.
    df_schedules : Dataframe on schedules from NFL FastR.
    df_draft_picks : Dataframe on draft picks from NFL FastR.
    df_rosters : Dataframe of rosters for current year from NFL FastR.
    df_injuries : Dataframe of injuries from NFL FastR.
    df_contracts : Dataframe of contracts from from NFL FastR.
    df_ids : Dataframe of ID joins back to 3rd party apps from NFL FastR.
    """
    #Import all the data needed for the model
    current_year = pd.Timestamp.now().year + 1
    years_use = list(np.arange(1999,current_year))
    injuries_year_use = list(np.arange(2009,current_year))
    try:
        df_weekly_stats = nfl.import_weekly_data(years=years_use)
    except HTTPError:
        print("Removing last year from weekly stats.")
        df_weekly_stats = nfl.import_weekly_data(years=years_use[:-1])
    print("Done loading weekly data.")
    try:
        df_schedules = nfl.import_schedules(years=years_use)
    except HTTPError:
        print("Removing last year from schedules.")
        df_schedules = nfl.import_schedules(years=years_use[:-1])
    print("Done loading schedules.")
    try:
        df_draft_picks = nfl.import_draft_picks(years=years_use)
    except HTTPError:
        print("Removing last year from draft picks.")
        df_draft_picks = nfl.import_draft_picks(years=years_use[:-1])
    print("Done loading draft picks.")
    try:
        df_rosters = nfl.import_rosters(years=years_use)
    except HTTPError:
        print("Removing last year from rosters.")
        df_rosters = nfl.import_rosters(years=years_use[:-1])
    print("Done loading roster.")
    try:
        df_injuries = nfl.import_injuries(years=injuries_year_use)
    except HTTPError:
        print("Removing last year from injuries.")
        df_injuries = nfl.import_injuries(years=injuries_year_use[:-1])
    print("Done loading injuries.")
    df_ids = nfl.import_ids()
    print("Done loading ids.")
    df_contracts = nfl.import_contracts()
    df_contracts = pd.merge(df_contracts,
                            df_ids[["name","gsis_id"]],how="inner",left_on="player",
                            right_on="name").drop("name",axis=1
                                                 ).rename({"gsis_id":"player_id",
                                                           "years":"number_years",
                                                           "value":"contract_value"},axis="columns")
    print("Done loading contracts.")
    df_contracts = df_contracts[["player","player_id","year_signed","number_years",
                                 "contract_value","apy","guaranteed"]]
    return df_weekly_stats,df_schedules,df_draft_picks,df_rosters,df_injuries,df_contracts,df_ids

In [ ]:
def define_injury(df,col,listvals,final_colname):
    """
    Cleans the injury list to capitalize and remove certain symbol characters and to specify specific injuries.
    Inputs:
    df : Dataframe of injury from NFL Fast R.
    col : Column containing injuries.
    listval : List of acceptable injuries that can be classified.
    final_colname : Final column  name of the injury column to use.
    
    Output:
    df - Injury dataframe with 'final_colname'.
    """
    for i,val in enumerate(listvals):
        if i == 0:
            df.loc[((~df[col].str.contains(",|/",na=False)) & 
                    (df[col].str.contains(val,na=False))),final_colname] = val
        else:
            df.loc[(((~df[col].str.contains(",|/",na=False)) & 
                     (df[col].str.contains(val,na=False))) & (df[final_colname].isnull())),final_colname] = val
    return df

In [ ]:
def clean_injuries(df_injuries,injury_list=list()):
    """
    Cleans the primary injury list on the injury df in the nfl dataset. 
    
    Inputs:
    df_injuries : Injury dataframe.
    injury_list : Optional - List of injuries - Default is an empty list that goes off a default list.
    
    Output: 
    df_injuries_final : Cleaned set of injuries to use for the feature engineering.
    """
    if len(injury_list) == 0:
        injury_list = ["KNEE", "ANKLE", "HAMSTRING", "SHOULDER", "FOOT", "HEAD", "GROIN", "BACK", "CALF", 
                       "ILLNESS", "HIP", "NECK", "QUADRICEP", "THIGH", "NOT INJURY RELATED", "HAND", "ELBOW", 
                       "RIB", "CHEST", "WRIST", "ARM", "ABDOMEN", "OTHER"]    
    else:
        injury_list = injury_list
    df_injuries = df_injuries[df_injuries["game_type"]=="REG"]
    df_injuries["report_primary_injury"] = df_injuries["report_primary_injury"].str.upper()
    df_injuries = define_injury(df_injuries,"report_primary_injury",injury_list,"injury_report_use")
    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                      ((df_injuries["report_primary_injury"].str.contains("TRAPEZIUS",na=False)) |
                       (df_injuries["report_primary_injury"].str.contains("STERNOCLAVICULAR",na=False)) |
                       (df_injuries["report_primary_injury"].str.contains("COLLARBONE",na=False)))) & 
                     (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "SHOULDER"
    
    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("TOE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("ACHILLE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("HEEL",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("FEET",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("ARCH",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "FOOT"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("CONCUSSION",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("EYE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("JAW",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("NOSE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("EAR",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("MOUTH",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("FACIAL",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("CHEEK",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("TOOTH",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("TEETH",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("CHIN",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "HEAD"
    
    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("GLUTE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("PELVIS",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("STINGER",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("BUTTOCKS",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "GROIN"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("HERNIA",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("TAILBONE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("LUMBAR",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "BACK"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("INFECTION",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("COVID",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("MIGRAINE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("APPENDICITIS",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("FLU",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "ILLNESS"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("OBLIQUE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("KIDNEY",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("STOMACH",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("LIVER",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("APPENDIX",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("ADDUCTOR",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("SPLEEN",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("QBLIQUE",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "HIP"
    
    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    (df_injuries["report_primary_injury"].str.contains("THROAT",na=False))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "NECK"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("SHIN",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("FIBULA",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("LEG",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("TIBIA",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("QUAD",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "QUADRICEP"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("THUMB",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("FINGER",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "HAND"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("PECTORAL",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("LUNG",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("CARDIAC",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("BREATH",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("ARRHYTHMIA",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("PLEXUS",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "CHEST"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("BICEP",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("TRICEP",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "ARM"

    df_injuries.loc[(((~df_injuries["report_primary_injury"].str.contains(",|/",na=False)) & 
                    ((df_injuries["report_primary_injury"].str.contains("CORE",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("ABDOMINAL",na=False)) |
                    (df_injuries["report_primary_injury"].str.contains("CRAMPS",na=False)))) & 
                    (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "ABDOMEN"
    
    df_injuries.loc[((df_injuries["report_primary_injury"].str.contains(",|/",na=False)) &
                     (df_injuries["report_primary_injury"].notnull()) & 
                     (df_injuries["injury_report_use"].isnull())),"injury_report_use"] = "MULTIPLE"
    df_injuries_use = df_injuries[df_injuries["injury_report_use"].notnull()]
    df_injuries_final = df_injuries_use[["season","game_type","team","week","gsis_id","report_status",
                                         "practice_status","injury_report_use"]]
    return df_injuries_final

In [ ]:
def split_players(df_weekly_stats,positions=["QB","RB","WR","TE"]):
    """
    Split players up between their resppective positions. Also calculates the prior week scores and week numbers.
    Inputs:
    df_weekly_stats : Dataframe of weekly stats for each player from NFL Fast R.
    positions : Optional - List of posiitons considered - Default opton is QB, RB, WR and TE.
    
    Outputs:
    df_qbs : A dataframe only containing QBs weekly data.
    df_rbs : A dataframe only containing RBs weekly data.
    df_wrs : A dataframe only containing WRs weekly data.
    df_tes : A dataframe only containing TEs weekly data.
    """
    qb_cols_keep = ["player_id","position","position_group","recent_team","season_type","completions","attempts",
                    "passing_yards","passing_tds","interceptions","sacks","sack_yards","sack_fumbles",
                    "sack_fumbles_lost","passing_air_yards","passing_yards_after_catch","passing_first_downs",
                    "passing_epa","passing_2pt_conversions","pacr","dakota","carries","rushing_yards","rushing_tds",
                    "rushing_fumbles","rushing_fumbles_lost","rushing_first_downs","rushing_epa",
                    "rushing_2pt_conversions","next_week","next_season","shifted_fantasy_points",
                    "shifted_fantasy_points_ppr","last_weeks_fantasy_points","last_weeks_fantasy_points_ppr"]
    rest_cols_keep = ["player_id","position","position_group","recent_team","season_type","carries","rushing_yards",
                      "rushing_tds","rushing_fumbles","rushing_fumbles_lost","rushing_first_downs","rushing_epa",
                      "rushing_2pt_conversions","receptions","targets","receiving_yards","receiving_tds",
                      "receiving_fumbles","receiving_fumbles_lost","receiving_air_yards","receiving_yards_after_catch",
                      "receiving_first_downs","receiving_epa","receiving_2pt_conversions","racr","target_share",
                      "air_yards_share","wopr","special_teams_tds","next_week","next_season","shifted_fantasy_points",
                      "shifted_fantasy_points_ppr","last_weeks_fantasy_points","last_weeks_fantasy_points_ppr"]
    
    df_weekly_stats = df_weekly_stats[df_weekly_stats["season_type"]=="REG"]
    df_weekly_stats["next_week"] = np.where(((df_weekly_stats["week"]<17) &
                                             (df_weekly_stats["season"]<2021)),df_weekly_stats["week"] + 1,np.nan)
    df_weekly_stats["next_week"] = np.where(((df_weekly_stats["week"]==17) & (df_weekly_stats["season"]<2021) & 
                                             (df_weekly_stats["next_week"].isnull())),1,
                                            df_weekly_stats["next_week"])
    df_weekly_stats["next_week"] = np.where(((df_weekly_stats["week"]<18) & (df_weekly_stats["season"]>=2021) & 
                                             (df_weekly_stats["next_week"].isnull())),df_weekly_stats["week"]+1,
                                            df_weekly_stats["next_week"])
    df_weekly_stats["next_week"] = np.where(((df_weekly_stats["week"]==18) & (df_weekly_stats["season"]>=2021) &
                                             (df_weekly_stats["next_week"].isnull())),1,
                                            df_weekly_stats["next_week"])
    
    df_weekly_stats["next_season"] = np.where(((df_weekly_stats["week"]<17) &
                                             (df_weekly_stats["season"]<2021)),df_weekly_stats["season"],np.nan)
    df_weekly_stats["next_season"] = np.where(((df_weekly_stats["week"]==17) & (df_weekly_stats["season"]<2021) & 
                                             (df_weekly_stats["next_season"].isnull())),
                                              df_weekly_stats["season"] + 1,df_weekly_stats["next_season"])
    df_weekly_stats["next_season"] = np.where(((df_weekly_stats["week"]<18) & (df_weekly_stats["season"]>=2021) & 
                                             (df_weekly_stats["next_season"].isnull())),df_weekly_stats["season"],
                                            df_weekly_stats["next_season"])
    df_weekly_stats["next_season"] = np.where(((df_weekly_stats["week"]==18) & (df_weekly_stats["season"]>=2021) &
                                             (df_weekly_stats["next_season"].isnull())),
                                              df_weekly_stats["season"] + 1,df_weekly_stats["next_season"])

    df_weekly_stats["recent_team"] = np.where(((df_weekly_stats["recent_team"]=="LA") & 
                                      (df_weekly_stats["next_season"]<=2015)),"STL",df_weekly_stats["recent_team"])
    df_weekly_stats["recent_team"] = np.where(((df_weekly_stats["recent_team"]=="LAC") & 
                                      (df_weekly_stats["next_season"]<=2016)),"SD",df_weekly_stats["recent_team"])
    df_weekly_stats["recent_team"] = np.where(((df_weekly_stats["recent_team"]=="LV") & 
                                      (df_weekly_stats["next_season"]<=2019)),"OAK",df_weekly_stats["recent_team"])
    df_weekly_stats = df_weekly_stats.reset_index().drop(["index","level_0"],axis=1,errors="ignore")
    df_weekly_stats["shifted_fantasy_points"] = df_weekly_stats.sort_values(
        by=["player_id","season","week"],ascending=True).groupby("player_id")["fantasy_points"].shift(-1)
    df_weekly_stats["shifted_fantasy_points_ppr"] = df_weekly_stats.sort_values(
        by=["player_id","season","week"],ascending=True).groupby("player_id")["fantasy_points_ppr"].shift(-1)
    df_weekly_stats["last_weeks_fantasy_points"] = df_weekly_stats["fantasy_points"]
    df_weekly_stats["last_weeks_fantasy_points_ppr"] = df_weekly_stats["fantasy_points_ppr"]
    df_weekly_stats = df_weekly_stats.drop(["week","season","fantasy_points","fantasy_points_ppr"],axis=1)
    df_qbs = df_weekly_stats[(df_weekly_stats["position_group"]=="QB")]
    df_rbs = df_weekly_stats[(df_weekly_stats["position_group"]=="RB")]
    df_wrs = df_weekly_stats[(df_weekly_stats["position_group"]=="WR")]
    df_tes = df_weekly_stats[(df_weekly_stats["position_group"]=="TE")]
    df_qbs = df_qbs[qb_cols_keep]
    df_rbs = df_rbs[rest_cols_keep]
    df_wrs = df_wrs[rest_cols_keep]
    df_tes = df_tes[rest_cols_keep]
    return df_qbs,df_rbs,df_wrs,df_tes

In [ ]:
def calc_rolling_average(df,cols_rolling,id_col,rolling_params=list(np.arange(2,11))):
    """
    Calculates the rolling average of a particular column for a given player.
    
    Inputs:
    df : Dataframe of interest.
    cols_rolling : List of columns to perform the rolling averages on.
    id_col : ID column to group by.
    rolling_params : Optional - List of number of prior games to include - default is making features for 
    the averaged values of the last 2-11 games.
    
    Output:
    df : Dataframe containing the rolling average feature of interest with the calculated last rolling_params games.
    """
    for game in rolling_params:
        for col in cols_rolling:
            df["avg_"+col+"_last_"+str(game)+"_games"] = df.groupby(
                id_col)[col].rolling(window=game,min_periods=1).mean().reset_index(drop=True)
    return df

In [ ]:
def feature_engineering(df_weekly,df_rosters,df_injuries,df_schedules,df_contracts,position):
    """
    Performs all of the feature engineering needed for the model.
    
    Inputs:
    df_weekly : Dataframe of weekly data.
    df_rosters : Dataframe of current roster for all teams.
    df_injuries : Dataframe of cleaned injuries for all players.
    df_schedules : Dataframe of current schedule for each team.
    df_contracts : Dataframe of all historical contracts for each player.
    position : Position of players of interest.
    
    Output:
    df_combine_merge : Dataframe that is feature engineered and ready for model prediction for a given position.
    """
    injury_col_dict = {"report_status":"Active",
                       "practice_status":"Active",
                       "current_injury":"Healthy",
                       "number_times_on_injury_report_season":0,
                       "number_times_out_play_status_season":0,
                       "number_times_out_practice_status_season":0}
    if position == "QB":
        combine_list = ["passing_yards","passing_tds","interceptions","rushing_yards","rushing_tds",
                        "sack_fumbles","rushing_fumbles_lost"]
    else:
        combine_list = ["carries","rushing_yards","rushing_tds","rushing_fumbles","rushing_fumbles_lost",
                        "rushing_first_downs","rushing_epa","rushing_2pt_conversions","receptions",
                        "receiving_yards","receiving_fumbles","receiving_air_yards","receiving_yards_after_catch",
                        "receiving_first_downs","receiving_2pt_conversions"]
    df_weekly = df_weekly.reset_index().drop(["index","level_0"],axis=1,errors="ignore")
    df_injuries = df_injuries.reset_index().drop(["index","level_0"],axis=1,errors="ignore")
    df_rosters = df_rosters.reset_index().drop(["index","level_0"],axis=1,errors="ignore")
    df_schedules = df_schedules.reset_index().drop(["index","level_0"],axis=1,errors="ignore")
    df_weekly["counter"] = np.arange(0,len(df_weekly))
    df_weekly["number_of_games_played"] = df_weekly.sort_values(by=["player_id","next_season","next_week"]
                                                     ).groupby(["player_id"])["counter"].cumcount() + 1
    df_weekly["games_with_team"] = df_weekly.sort_values(by=["player_id","next_season","next_week","next_season"]
                                            ).groupby(["player_id","recent_team"])["next_season"].cumcount() + 1
    df_weekly.drop("counter",axis=1,inplace=True)
    df_first_game = df_weekly.sort_values(by=["player_id","next_season","next_week"]
                                   ,ascending=True).groupby("player_id"
                                                           ).first().reset_index()[["player_id","next_season",
                                                                                    "recent_team","next_week",
                                                                                    "last_weeks_fantasy_points",
                                                                                    "last_weeks_fantasy_points_ppr"]
                                                                                  ].rename(
        {"last_weeks_fantasy_points":"shifted_fantasy_points",
         "last_weeks_fantasy_points_ppr":"shifted_fantasy_points_ppr"},axis="columns")

    df_first_game["next_season"] = np.where(df_first_game["next_week"]!=1,df_first_game["next_season"],
                                            df_first_game["next_season"]-1)
    df_first_game.loc[df_first_game["next_week"]!=1,"next_week"] = df_first_game["next_week"] - 1
    df_first_game.loc[(df_first_game["next_week"]==1) & (df_first_game["next_season"]<=2021),"next_week"] = 17
    df_first_game.loc[(df_first_game["next_week"]==1) & (df_first_game["next_season"]>2021),"next_week"] = 18
    df_combine = pd.concat([df_first_game,df_weekly],axis=0).sort_values(
        by=["player_id","next_season","next_week"],ascending=True).reset_index(drop=True)
    #df_combine = calc_rolling_average(df_combine,combine_list,"player_id","next_week")
    df_combine = calc_rolling_average(df_combine,combine_list,"player_id")
    df_draft_picks = df_rosters[df_rosters["entry_year"].notnull()].sort_values(
        by=["player_id","season"],ascending=False).drop_duplicates(subset=["player_id"],keep="first")
    df_draft_use = df_draft_picks[["player_id","entry_year","rookie_year","draft_club","draft_number"]]
    df_combine = pd.merge(df_combine,df_draft_use,how="left",on="player_id")
    df_injuries["season"] = df_injuries["season"].astype("Int64")
    df_injuries["week"] = df_injuries["week"].astype("Int64")
    df_play_out = df_injuries[df_injuries["report_status"]=="Out"][["season","week","gsis_id"]]
    df_practice_out = df_injuries[((df_injuries["practice_status"]=="Did Not Participate In Practice") | 
                                   (df_injuries["practice_status"]=="Out (Definitely Will Not Play)")) & 
                                  (df_injuries["practice_status"].notnull())][["season","week","gsis_id"]]
    df_injuries["number_times_on_injury_report_season"] = df_injuries.sort_values(
        by=["gsis_id","season","week"],ascending=True).groupby(["gsis_id","season"])["week"].cumcount() + 1
    df_play_out["number_times_out_play_status_season"] = df_play_out.sort_values(
        by=["gsis_id","season","week"],ascending=True).groupby(["gsis_id","season"])["week"].cumcount() + 1
    df_practice_out["number_times_out_practice_status_season"] = df_practice_out.sort_values(
        by=["gsis_id","season","week"],ascending=True).groupby(["gsis_id","season"])["week"].cumcount() + 1
    df_combine["next_season"] = df_combine["next_season"].astype("Int64")
    df_combine["next_week"] = df_combine["next_week"].astype("Int64")
    df_combine = pd.merge(df_combine,df_injuries.drop(["game_type","team"],axis=1).rename(
        {"injury_report_use":"current_injury"},axis="columns"),how="left",
                          left_on=["player_id","next_season","next_week"],right_on=["gsis_id","season","week"]
                         ).drop(["gsis_id","season","week"],axis=1)
    df_combine = pd.merge(df_combine,df_play_out,how="left",left_on=["player_id","next_season","next_week"],
                          right_on=["gsis_id","season","week"]).drop(["gsis_id","season","week"],axis=1)
    df_combine = pd.merge(df_combine,df_practice_out,how="left",left_on=["player_id","next_season","next_week"],
                          right_on=["gsis_id","season","week"]).drop(["gsis_id","season","week"],axis=1)
    df_combine["entry_year"] = df_combine["entry_year"].astype(float).astype("Int64")
    df_combine["years_played"] = (df_combine["next_season"] - df_combine["entry_year"]).astype("Int64")
    df_combine["draft_team"] = np.where(df_combine["draft_club"]==df_combine["recent_team"],True,False)
    df_combine["draft_number"] = df_combine["draft_number"].astype(float).astype("Int64")
    df_combine["drafted_high"] = np.where(((df_combine["draft_number"].notnull())
                                           & (df_combine["draft_number"]<90)),1,0)
    df_combine["is_rookie"] = np.where(((df_combine["entry_year"]==df_combine["next_season"]) & 
                                        (df_combine["entry_year"].notnull())),1,0)    
    for k,v in injury_col_dict.items():
        df_combine[k].fillna(v,inplace=True)   
    df_schedules_id = df_schedules[["game_id","season","week","home_team","away_team"]]
    df_combine["last_season"] = np.where(df_combine["next_week"]!=1,df_combine["next_season"],
                                         df_combine["next_season"]-1)
    df_combine.loc[df_combine["next_week"]!=1,"last_week"] = df_combine["next_week"] - 1
    df_combine.loc[(df_combine["next_week"]==1) & (df_combine["next_season"]<=2021),"last_week"] = 17
    df_combine.loc[(df_combine["next_week"]==1) & (df_combine["next_season"]>2021),"last_week"] = 18
    df_combine = pd.merge(df_combine,df_schedules_id.drop("away_team",axis=1),how="left",
                          left_on=["last_season","last_week","recent_team"],
                          right_on=["season","week","home_team"]).drop(["season","week"],axis=1)

    df_combine = pd.merge(df_combine,df_schedules_id.drop("home_team",axis=1),how="left",
                          left_on=["last_season","last_week","recent_team"],
                          right_on=["season","week","away_team"]).drop(["season","week"],axis=1)
    df_combine["last_week_game_id"] = np.where(df_combine["game_id_x"].notnull(),
                                               df_combine["game_id_x"],df_combine["game_id_y"])
    df_combine["last_week_game_id"] = np.where(df_combine["last_week_game_id"].isnull(),"Bye",
                                               df_combine["last_week_game_id"])
    df_combine.loc[(df_combine["home_team"].notnull()),"last_week_home_away"] = "Home"
    df_combine.loc[(df_combine["away_team"].notnull()),"last_week_home_away"] = "Away"
    df_combine.loc[((df_combine["home_team"].isnull()) & (df_combine["away_team"].isnull())),
                   "last_week_home_away"] = "Bye"
    df_combine.drop(["game_id_x","game_id_y","home_team","away_team"],axis=1,inplace=True)
    df_combine = pd.merge(df_combine,df_schedules[["game_id","weekday","gametime","away_team","away_score",
                                                    "home_team","home_score","location","overtime","roof",
                                                    "surface","temp","wind","stadium"]],
                          how="left",left_on="last_week_game_id",right_on="game_id"
                         ).drop(["last_week_game_id","game_id"],axis=1).rename(
        {"weekday":"last_week_weekday","gametime":"last_week_gametime","away_team":"last_week_away_team",
         "away_score":"last_week_away_score","home_team":"last_week_home_team","home_score":"last_week_home_score",
         "location":"last_week_location","overtime":"last_week_overtime","roof":"last_week_roof",
         "surface":"last_week_surface","temp":"last_week_temp","wind":"last_week_wind",
         "stadium":"last_week_stadium"},axis="columns")    
    df_combine = pd.merge(df_combine,df_schedules_id.drop("away_team",axis=1),how="left",
                          left_on=["next_season","next_week","recent_team"],
                          right_on=["season","week","home_team"]).drop(["season","week"],axis=1)

    df_combine = pd.merge(df_combine,df_schedules_id.drop("home_team",axis=1),how="left",
                          left_on=["next_season","next_week","recent_team"],
                          right_on=["season","week","away_team"]).drop(["season","week"],axis=1)
    df_combine["next_week_game_id"] = np.where(df_combine["game_id_x"].notnull(),
                                               df_combine["game_id_x"],df_combine["game_id_y"])
    df_combine["next_week_game_id"] = np.where(df_combine["next_week_game_id"].isnull(),"Bye",
                                               df_combine["next_week_game_id"])
    df_combine.loc[(df_combine["home_team"].notnull()),"next_week_home_away"] = "Home"
    df_combine.loc[(df_combine["away_team"].notnull()),"next_week_home_away"] = "Away"
    df_combine.loc[((df_combine["home_team"].isnull()) & (df_combine["away_team"].isnull())),
                   "next_week_home_away"] = "Bye"
    df_combine.drop(["game_id_x","game_id_y","home_team","away_team"],axis=1,inplace=True)
    df_combine = pd.merge(df_combine,df_schedules[["game_id","weekday","gametime","away_team",
                                                   "home_team","location","roof","surface","stadium"]],
                          how="left",left_on="next_week_game_id",right_on="game_id"
                         ).drop(["next_week_game_id","game_id"],axis=1).rename(
        {"weekday":"next_week_weekday","gametime":"next_week_gametime","away_team":"next_week_away_team",
         "home_team":"next_week_home_team","location":"next_week_location","roof":"next_week_roof",
         "surface":"next_week_surface","stadium":"next_week_stadium"},axis="columns")
    df_combine_merge = pd.merge(df_combine,df_contracts,how="left",on="player_id")
    df_combine_merge = df_combine_merge[df_combine_merge["year_signed"]<=df_combine_merge["next_season"]]
    df_combine_merge = df_combine_merge.sort_values(by=["player_id","next_season","next_week","year_signed"],
                                                    ascending=[True,True,True,False]).drop_duplicates(
        subset=["player_id","next_season","next_week"],keep="first")
    df_combine_merge["year_signed"].fillna(df_combine_merge["next_season"],inplace=True)
    df_combine_merge["number_years"].fillna(1,inplace=True)
    df_combine_merge["contract_value"].fillna(0.36,inplace=True)
    df_combine_merge["apy"].fillna(0.36,inplace=True)
    df_combine_merge["guaranteed"].fillna(0,inplace=True)
    df_combine_merge["number_of_seasons_since_last_contract"] = df_combine_merge["next_season"] - \
    df_combine_merge["year_signed"]
    return df_combine_merge

In [ ]:
def make_predictions(model_path,features_df,featureset_df,position,df_current_roster,
                     ids_keep=["player_id","recent_team","next_season","next_week","report_status"]):
    """
    Makes predictions for a given position for predicted number of points that a player will score in a given week.
    
    Inputs:
    model_path : Path where the trained catboost model is located.
    features : Dataframe containing all of the relevant features used for modeling.
    featureset_df : Dataframe containing all of the players and features (including IDs for modeling).
    position : Position of interest.
    df_current_roster : Dataframe of current roster.
    ids_keep : List of columns that are kept but are not features and predicted values.
    
    Output:
    X_pred_ids : Dataframe of players and the predicted values that they would score.
    """
    if position == "QB":
        max_val = 45.92
    elif position == "RB":
        max_val = 59.5
    elif position == "WR":
        max_val = 57.9
    elif position == "TE":
        max_val = 45.0
    else:
        raise("Wrong position")
    model = CatBoostRegressor().load_model(model_path)
    featureset_df_use = featureset_df.sort_values(by=["player_id","next_season","next_week"],
                                                  ascending=False).drop_duplicates(subset=["player_id"],keep="first")
    featureset_df_use = featureset_df_use[(featureset_df_use["next_week_home_away"]!="Bye")]
    featureset_df_use.drop("recent_team",axis=1,inplace=True)
    featureset_df_use = pd.merge(featureset_df_use,df_current_roster[["player_id","team"]]
                                 ,how="left",on="player_id").rename({"team":"recent_team"},axis="columns")
    X_pred = featureset_df_use[features_df["column_name"].tolist()]
    X_pred_ids = featureset_df_use[ids_keep]
    X_pred["entry_year"] = X_pred["entry_year"].astype(str)
    X_pred["rookie_year"] = X_pred["rookie_year"].astype(str)
    X_pred["draft_club"].fillna("Undrafted",inplace=True)
    X_pred.drop(["last_week","last_season"],axis=1,inplace=True,errors="ignore")

    categorical_features_names = ["position","position_group","season_type","recent_team","draft_club",
                                  "entry_year","rookie_year","report_status","practice_status",
                                  "current_injury","last_week_home_away","last_week_weekday",
                                  "last_week_gametime","last_week_away_team","last_week_home_team",
                                  "last_week_location","last_week_roof","last_week_surface","last_week_stadium",
                                  "next_week_home_away","next_week_weekday","next_week_gametime",
                                  "next_week_away_team","next_week_home_team","next_week_location",
                                  "next_week_roof","next_week_surface","next_week_stadium"]
    for col in X_pred.columns:
        if col in categorical_features_names:
            X_pred[col].fillna("N/A",inplace=True)
    for col in X_pred.columns:
        if X_pred[col].dtype=="Int64":
            X_pred[col].fillna(0,inplace=True)
    X_pred_pool = Pool(X_pred,cat_features=categorical_features_names)
    X_pred_ids["predicted_log_value"] = model.predict(X_pred_pool)
    if position == "QB":
        X_pred_ids["predicted_value"] = round((-(np.exp(X_pred_ids["predicted_log_value"]))+max_val),2)
    else:
        X_pred_ids["predicted_value"] = round((-(np.exp(X_pred_ids["predicted_log_value"]))+max_val),1)
    X_pred_ids = pd.merge(X_pred_ids,df_current_roster[["player_id","player_name"]],
                          how="left",on="player_id").drop("predicted_log_value",axis=1)
    X_pred_ids = X_pred_ids[X_pred_ids["player_name"].notnull()]
    X_pred_ids["predicted_value"] = np.where(X_pred_ids["report_status"]=="Out",0,X_pred_ids["predicted_value"])
    X_pred_ids = pd.merge(X_pred_ids,df_current_roster[["player_id","status"]],how="left",on="player_id")
    X_pred_ids["predicted_value"] = np.where(X_pred_ids["status"]!="ACT",0,X_pred_ids["predicted_value"])
    return X_pred_ids

In [ ]:
def determine_fantasy_engine(df_ids):
    """
    Determines which platform the user is using for their fantasy football purposes. Currently supports ESPN,
    Sleeper, and Yahoo.
    
    Input:
    df_ids : Dataframe of join IDs from NFL FastR.
    
    Outputs:
    df_merge : Dataframe of players that are in the user's team and free agents. 
    df_slots : Dataframe of positions available in the user's league and quantity needed for starting. 
    """
    engine = input("Please enter the fantasy football engine used. Currently supported: 'espn','sleeper' or 'yahoo'")
    if engine == "espn":
        df_merge,df_slots = access_espn_players_and_roster_limits(df_ids)
    elif engine == "sleeper":
        df_merge,df_slots = access_sleeper_players_and_roster_limits(df_ids)
    elif engine == "yahoo":
        df_merge,df_slots = access_yahoo_players_and_roster_limits(df_ids)
    else:
        print(engine + " is currently unsupported.")
        df_merge = pd.DataFrame()
        df_slots = pd.DataFrame()
    return df_merge,df_slots

In [ ]:
def access_espn_players_and_roster_limits(df_ids):
    """
    API connector to access players and league roster settings from ESPN.
        
    Input:
    df_ids : Dataframe of join IDs from NFL FastR.
    
    Outputs:
    df_merge : Dataframe of players that are in the user's team and free agents in an ESPN league. 
    df_slots : Dataframe of positions available in the user's league and quantity needed for starting in an 
    ESPN league. 
    """
    league_id = input("Please enter the league id. The league id can be found in the URL of the league homepage.")
    league_id = int(league_id)
    year = input("Please enter the season.")
    year = int(year)
    espn_s2 = input("Please enter the espn_s2 authentication. Instructions can be found in the following link: https://cran.r-project.org/web/packages/ffscrapr/vignettes/espn_authentication.html")
    swid = input("Please enter the swid authentication. Instructions can be found in the following link: https://cran.r-project.org/web/packages/ffscrapr/vignettes/espn_authentication.html")
    team_id = input("Please enter the team id. The team id can be found in the URL of the league homepage.")
    team_id = int(team_id)
    league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)
    team = league.get_team_data(team_id)
    free_agents = league.free_agents(size=1000)
    player_name_list = []
    player_id_list = []
    for player in free_agents:
        player_name_list.append(player.name)
        player_id_list.append(player.playerId)
    
    for player in team.roster:
        player_name_list.append(player.name)
        player_id_list.append(player.playerId)
    
    espn_player_df = pd.concat([pd.Series(player_name_list,name="player_name"),
                                pd.Series(player_id_list,name="espn_id")],axis=1)
    df_merge = pd.merge(espn_player_df,df_ids,how="inner",on="espn_id")
    df_slots = pd.DataFrame.from_dict(league.espn_request.get_league()["settings"]
                                  ["rosterSettings"]["lineupSlotCounts"],orient="index").reset_index()
    df_slots.rename({"0":"allowance",0:"allowance"},axis="columns",inplace=True)
    df_slots["index"] = df_slots["index"].astype(str)
    position_df = pd.DataFrame.from_dict(POSITION_MAP,orient="index").reset_index()
    position_df["index"] = position_df["index"].astype(str)
    position_df.rename({"0":"position",0:"position"},axis="columns",inplace=True)
    df_slots = pd.merge(df_slots,position_df,how="inner",on="index").drop("index",axis=1)
    df_slots = df_slots[df_slots["allowance"]>0]
    return df_merge,df_slots

In [ ]:
def access_sleeper_players_and_roster_limits(df_ids):
    """
    API connector to access players and league roster settings from Sleeper.
        
    Input:
    df_ids : Dataframe of join IDs from NFL FastR.
    
    Outputs:
    df_merge : Dataframe of players that are in the user's team and free agents in a Sleeper league. 
    df_slots : Dataframe of positions available in the user's league and quantity needed for starting in a 
    Sleeper league. 
    """
    username = input("Please enter the sleeper username, not team name.")
    league_id = input("Please enter the league id. This can be found when you open league settings --> general and scroll to the bottom.")
    league_id = int(league_id)
    season = input("Please enter the season.")
    season = int(season)
    account_id = User.get_user(username)["user_id"]
    df_teams = pd.DataFrame(Leagues.get_rosters(league_id))
    other_players_list = []
    players_other_team = df_teams[df_teams["owner_id"]!=account_id].reset_index()["players"]
    for index,row in players_other_team.iteritems():
        for value in row:
            if value.isdigit():
                other_players_list.append(float(value))
    df_pool = pd.DataFrame(Players.get_all_players()).rename({"0":"sleeper_id",0:"sleeper_id"},axis="columns")
    df_pool = df_pool[df_pool["sleeper_id"].str.isdigit()]
    df_pool["sleeper_id"] = df_pool["sleeper_id"].astype(float)
    df_keep = df_pool[~(df_pool["sleeper_id"].isin(other_players_list))]
    df_ids_use = df_ids[df_ids["sleeper_id"].notnull()]
    df_merge = pd.merge(df_keep,df_ids_use,how="inner",on="sleeper_id").rename({"name":"player_name"},axis="columns")
    position_df = pd.Series(Leagues.get_league(league_id)["roster_positions"],name="positions")
    df_slots = position_df.value_counts(dropna=False).reset_index().rename({"positions":"allowance","index":"position"},axis="columns")
    return df_merge,df_slots

In [ ]:
def access_yahoo_players_and_roster_limits(df_ids,sport="nfl"):
    """
    API connector to access players and league roster settings from Yahoo.
        
    Input:
    df_ids : Dataframe of join IDs from NFL FastR.
    sport : Optional - sport of interst - Set at nfl
    
    Outputs:
    df_merge : Dataframe of players that are in the user's team and free agents in a Yahoo league. 
    df_slots : Dataframe of positions available in the user's league and quantity needed for starting in a 
    Yahoo league. 

    """
    path_of_key = input("Please provide the path of the authentication file. It should be a .json file. Please see https://github.com/mattdodge/yahoofantasy under Installation and authentication for more details.")
    league_id = input("Please enter the league id. This can be found in the Scoring & Settings section of your fantasy football home page.")
    team_name = input("Please enter the team name.")
    sc = OAuth2(None, None, from_file=path_of_key)
    gm = yfa.Game(sc,sport)
    lg = gm.to_league(gm.game_id() + ".l." + str(league_id))
    team_use = lg.get_team(team_name)
    team_df = pd.DataFrame()
    team_list = team_use[team_name].roster()
    for team in team_list:
        team_df = pd.concat([team_df,pd.DataFrame(team)],axis=0)
    team_df = team_df.drop_duplicates(subset="name",keep="first")
    free_agent_df = get_yahoo_free_agents(lg)
    combined_df = pd.concat([team_df,free_agent_df],axis=0)
    combined_df_use = combined_df.drop_duplicates(subset="player_id").rename(
        {"player_id":"yahoo_id"},axis="columns")["yahoo_id"]
    df_ids_use = df_ids[df_ids["yahoo_id"].notnull()]
    df_merge = pd.merge(combined_df_use,df_ids,how="inner",on="yahoo_id").rename(
        {"name":"player_name"},axis="columns")
    df_slots = pd.DataFrame(lg.positions()).T.reset_index().rename(
        {"index":"position","count":"allowance"},axis="columns")[["position","allowance"]]
    return df_merge,df_slots

def get_yahoo_free_agents(lg,positions=["QB","RB","WR","TE"]):
    """
    Get all available free agents from Yahoo Fantasy Football for a given league.
    
    Inputs:
    lg : League class from Yahoo Fantasy Football.
    positions : Optional - list of available positions to grab free agents. Default is QB, RB, WR and TE.
    
    Output:
    df_final : Dataframe of all free agents available in the league of interst.
    """
    df_final = pd.DataFrame()
    for position in positions:
        df_free_agents = pd.DataFrame()
        position_list = lg.free_agents(position=position)
        position_df = pd.DataFrame()
        for record in position_list:
            position_df = pd.concat([position_df,pd.DataFrame(record)],axis=0)
        position_df = position_df.reset_index().drop("index",axis=1)
        position_df = position_df[position_df["eligible_positions"]==position]
        df_final = pd.concat([df_final,position_df],axis=0)
    return df_final

In [ ]:
def determine_optimal_roster(df_qb_preds,df_rb_preds,df_wr_preds,df_te_preds,df_options,df_max):
    """
    Determines the optimal roster for a given league based on availability of the current roster and 
    free agent pool. Currently only works for QB, RB, WR and TE. Future will include DEF/ST and K.
    
    Inputs:
    df_qb_preds : Dataframe of all predictions for the QBs that were made.
    df_rb_preds : Dataframe of all predictions for the RBs that were made.
    df_wr_preds : Dataframe of all predictions for the WRs that were made.
    df_te_preds : Dataframe of all predictions for the TEs that were made.
    df_options : Dataframe of all available players that the user can use (either using free agency or with their own
    roster).
    df_max : Dataframe containing the maximum number of players that a league can have per position. 
    
    Outputs:
    df_best_qb : Dataframe containing the best QBs based on fantasy points - length is determined by 
    number of max starting players based on league setting.
    df_best_rb : Dataframe containing the best RBs based on fantasy points - length is determined by 
    number of max starting players based on league setting.
    df_best_wr : Dataframe containing the best WRs based on fantasy points - length is determined by 
    number of max starting players based on league setting.
    df_best_te : Dataframe containing the best TEs based on fantasy points - length is determined by 
    number of max starting players based on league setting.
    df_best_flex : Dataframe containing the best FLEX players based on fantasy points - length is determined by 
    number of max starting players based on league setting. 
    Flex is defined here any player that can be listed as a RB, WR or TE.
    """
    df_qbs = pd.merge(df_qb_preds,df_options.drop("player_name",axis=1),how="inner",left_on="player_id",
                      right_on="gsis_id").drop("gsis_id",axis=1)
    df_best_qb = df_qbs.sort_values(by="predicted_value",ascending=False).head(
        df_max[df_max["position"]=="QB"].reset_index()["allowance"][0])
    df_rbs = pd.merge(df_rb_preds,df_options.drop("player_name",axis=1),how="inner",left_on="player_id",
                      right_on="gsis_id").drop("gsis_id",axis=1)
    df_best_rb = df_rbs.sort_values(by="predicted_value",ascending=False).head(
        df_max[df_max["position"]=="RB"].reset_index()["allowance"][0])
    df_wrs = pd.merge(df_wr_preds,df_options.drop("player_name",axis=1),how="inner",left_on="player_id",
                      right_on="gsis_id").drop("gsis_id",axis=1)
    df_best_wr = df_wrs.sort_values(by="predicted_value",ascending=False).head(
        df_max[df_max["position"]=="WR"].reset_index()["allowance"][0])
    df_tes = pd.merge(df_te_preds,df_options.drop("player_name",axis=1),how="inner",left_on="player_id",
                      right_on="gsis_id").drop("gsis_id",axis=1)
    df_best_te = df_tes.sort_values(by="predicted_value",ascending=False).head(
        df_max[df_max["position"]=="TE"].reset_index()["allowance"][0])
    df_flex = pd.concat([df_rbs,df_wrs,df_tes],axis=0)
    df_flex = df_flex[(~(df_flex["player_id"].isin(df_best_rb["player_id"])) & 
                        ~(df_flex["player_id"].isin(df_best_wr["player_id"])) & 
                        ~(df_flex["player_id"].isin(df_best_te["player_id"])))]
    df_best_flex = df_flex.sort_values(by="predicted_value",ascending=False).head(
        df_max[(df_max["position"]=="RB/WR/TE") | (df_max["position"]=="FLEX") | 
               (df_max["position"]=="W/R/T")].reset_index()["allowance"][0])
    return df_best_qb,df_best_rb,df_best_wr,df_best_te,df_best_flex

In [ ]:
def print_optimal_roster(df,flex=False):
    """
    Prints the recommended roster based on the model.
    
    Inputs:
    df : Best dataframe roster of interst.
    flex : Optional - Boolean of True and False - Default value is False.
    
    Output:
    None
    """
    for index,row in df.iterrows():
        if flex == False:
            print("Best " + row["position"] + " is " + row["player_name"] + " and predicted to score " + \
                  str(row["predicted_value"]) + " points.")
        else:
            print("Best flex is " + row["player_name"] + " and predicted to score " + \
                  str(row["predicted_value"]) + " points.")
    return

In [ ]:
def print_player_predictions(df,player_name):
    """
    Prints out the prediction for a given player.
    
    Inputs:
    df : Dataframe for a position of interest.
    player_name : player name of interest
    
    Output:
    None
    """
    df_player = df[df["player_name"]==player_name]
    print(player_name + " is expected to score " + str(df_player.reset_index()["predicted_value"][0]) + " points.")
    return